# TFLite Interpreter

This code is based on `rl_evaluate_policy` package `print_returns_positions module`.

In [ ]:
import tensorflow as tf
from tf_agents.trajectories.time_step import TimeStep

from machine_learning_rl_training.src.rl_environment_single_agent import RaceSimulation
from machine_learning_rl_training.src.rl_evaluate_policy import convert_time_step, print_returns_positions

model_path = "/content/gdrive/MyDrive/RLF002/vse-002/exports/2023-05-21-final/nn_reinforcement_Shanghai_2019.tflite"
race_pars_file = "/content/racesim/input/parameters/pars_Shanghai_2019.ini"
mcs_pars_file = "/content/racesim/input/parameters/pars_mcs.ini"
vse_others = "basestrategy"

py_env = RaceSimulation(
    race_pars_file=race_pars_file,
    mcs_pars_file=mcs_pars_file,
    vse_type=vse_others,
    use_prob_infl=True,
    create_rand_events=True)

print(
  py_env.action_spec(),
  py_env.observation_spec(),
)

interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
# input_index = interpreter.get_input_details()[0]["index"]  # old model
input_index = interpreter.get_input_details()[2]["index"]  # new model
output_index = interpreter.get_output_details()[0]["index"]

In [ ]:
time_step = py_env.reset()
episode_return = 0.0

lap = 1
while not time_step.is_last():
    converted_time_step = convert_time_step(time_step)
    observation = converted_time_step.observation
    interpreter.set_tensor(2, converted_time_step.observation)
    interpreter.invoke()
    print(interpreter.get_tensor(output_index))
    # action_q = interpreter.get_tensor(output_index)[0].argmax()  # old model
    action_q = interpreter.get_tensor(output_index)[0]  # new model
    print(f"driver = {py_env.idx_driver}, lap = {lap}, action = {action_q}")
    time_step = py_env.step(action_q)
    episode_return += time_step.reward
    lap += 1

final_position = py_env.race.positions[py_env.race.get_last_compl_lap(py_env.idx_driver), py_env.idx_driver]
print(f"driver = {py_env.idx_driver}, final_position = {final_position}")